<ul>
<li><a href="#introduction">1 - Online Machine Learning</a></li>
<li><a href="#river">2 - River </a></li>
    <ul>
        <li><a href="#cluster">2.1 Clustering</a></li>
        <li><a href="#drift">2.2 Concept Drift</a></li>
                <li><a href="#pipe">2.3 Build Pipeline</a></li>
    </ul>

</ul>

<a id='introduction'></a>

**1. Online Machine Learning**

Online machine learning processes one data at a time which also means model is updated as data come in real or near-real time, on the other hand there is offline machine learning that works on batch or historical data. There is some platforms that allows us to online computation like:

* Apache Kafka for database
* Apache Flink for analytic 
* River for machine learning
* Parent Packages of River
  * Creme
  * Scikit-Multiflow

> Limitation of online machine learning
  *   Cant do vectoriztion
  *   less memory
  *   less unknown so limited resource avalaible in community and lack of good examples
  *   if there is historical or given dataset then batch processing more meaningful
  * may not be avaliable for some ml algorithms


> Goodness of online machine learning

  * requires low computational power
  * feedback is fast (for ex. recommendation)
  * non static learning
  * dont have to revisit past data
  * may able to identify and detect concept drift











<a id='river'></a>

**2. River**

River is python libraray that allows us to process online machine learning applications. River datasets are python dicts rather than more sophisticated types like nump array or pandas dataframe.

> Some Useful modules:
  * Datasets
  * Models
  * Pipelines
  * Cluster
  * Drift




In [2]:
import river
import pandas as pd
import river
from river import cluster
from river import stream
from river import datasets
from sklearn.metrics.cluster import adjusted_rand_score
from river.datasets import synth
import numpy as np
from river.drift import ADWIN
from river import metrics
from river import drift


In [2]:
def get_all_attributes(package):
    subpackages = []
    submodules = []
    for i in dir(package):
        if str(i) not in ["__all__", "__builtins__", "__cached__", "__doc__", "__file__", "__loader__", "__name__", "__package__", "__path__", "__pdoc__", "__spec__", "__version__"]:
            subpackages.append(i)
            res = [j for j in dir(eval("river.{}".format(i)))]
            submodules.append(res)
    df = pd.DataFrame(submodules)
    df = df.T
    df.columns = subpackages
    res_df = df.dropna()
    return res_df
river_df = get_all_attributes(river)

In [3]:
river_df.columns

Index(['base', 'cluster', 'datasets', 'stats', 'stream', 'utils'], dtype='object')

In [14]:
river_df[["cluster","datasets"]]

,cluster,datasets
0,CluStream,AirlinePassengers
1,DBSTREAM,Bananas
2,DenStream,Bikes
3,KMeans,ChickWeights
4,STREAMKMeans,CreditCard
5,TextClust,Elec2
6,__all__,HTTP
7,__builtins__,Higgs
8,__cached__,ImageSegments
9,__doc__,Insects


In [5]:
from river import datasets
dataset = datasets.Phishing()
for x,y in dataset:
  continue
print(x)

{'empty_server_form_handler': 1.0, 'popup_window': 0.5, 'https': 1.0, 'request_from_other_domain': 1.0, 'anchor_from_other_domain': 1.0, 'is_popular': 0.5, 'long_url': 0.0, 'age_of_domain': 0, 'ip_in_url': 0}


**2.1 Clustering**
<a id='cluster'></a>

* CluStream

* DBSTREAM

* DenStream

* KMeans

* STREAMKMeans

* TextClust

In [6]:
!pip freeze | grep river


river==0.14.0


In [15]:

X = [
     [1, 2],
     [1, 4],
     [1, 0],
     [4, 2],
     [4, 4],
     [4, 0],
     [-2, 2],
     [-2, 4],
     [-2, 0]
 ]

k_means = cluster.KMeans(n_clusters=3, halflife=0.4, sigma=3, seed=0)
metric = metrics.Silhouette()

for x, _ in stream.iter_array(X):
     k_means = k_means.learn_one(x)
     y_pred = k_means.predict_one(x)
     metric = metric.update(x, y_pred, k_means.centers)
     print(x,y_pred,metric)

{0: 1, 1: 2} 1 Silhouette
{0: 1, 1: 4} 1 Silhouette
{0: 1, 1: 0} 1 Silhouette
{0: 4, 1: 2} 1 Silhouette
{0: 4, 1: 4} 1 Silhouette
{0: 4, 1: 0} 1 Silhouette
{0: -2, 1: 2} 2 Silhouette
{0: -2, 1: 4} 2 Silhouette
{0: -2, 1: 0} 2 Silhouette


**2.2 Concept Drift**
<a id='drift'></a>

The concept drift problem means that probability distribution of the data changes unforeseenly and causes less accurate predictions over time. Batch processing techniques more likely to fail because the model already trained with different data But online learning methods continuously update themselves and they can detect concept drifting much earlier and may adapt themselves.

* ADWIN
* DDM
* EDDM
* HDDM_A
* HDDM_W
* KSWIN
* PageHinkley
* PeriodicTrigger

In [17]:

drift_detector = drift.ADWIN
drifts = []

for i, val in enumerate(stream):
    drift_detector.update(val)   # Data is processed one sample at a time
    if drift_detector.change_detected():
        # The drift detector indicates after each sample if there is a drift in the data
        print(f'Change detected at index {i}')
        drifts.append(i)

plot_data(dist_a, dist_b, dist_c, drifts)

TypeError: 'module' object is not iterable

In [4]:

adwin = ADWIN()

# Simulate a data stream composed by two data distributions
data_stream = np.concatenate((np.random.randint(2, size=1000),
                               np.random.randint(4, high=8, size=1000)))

# Update drift detector and verify if change is detected
print(np.mean(data_stream[0:1000]),np.mean(data_stream[1000:len(data_stream)]))
for i, val in enumerate(data_stream):
     in_drift, in_warning = adwin.update(val)
     if in_drift:
         print(f"Change detected at index {i}, input value: {val}")

0.516 5.544


TypeError: cannot unpack non-iterable ADWIN object

**2.3 Build Pipeline**
<a id='pipe'></a>

Chain a sequence of operations and warrant reproducibility. 
* first n − 1 steps
are transformers like scaler, bag of words, normalizer
* last step can be a regressor, a classifier, a clusterer, a transformer 


In [10]:
from river import linear_model, preprocessing
model = (preprocessing.StandardScaler() |
linear_model.LogisticRegression())

Run CluStream on Keystroke Data

In [28]:
df = datasets.Keystroke()
clustream = cluster.CluStream(
n_macro_clusters=51,
max_micro_clusters=60,
time_gap=3,
    seed=0,
    halflife=0.4
)
predictions ={}
actual = {}
i=0
for x, y in df:
    clustream = clustream.learn_one(x)
    predictions[i] = clustream.predict_one(x)
    actual[i] = y
    i = i+1

    


Calculate adjusted rand score 

In [20]:
t_dict={}
k=0
for i in sorted(set(actual.values())):
    t_dict[i] = k
    k= k+1
actual_clusters=[]
for i in actual.values():
    actual_clusters.append(t_dict[i])
adjusted_rand_score(actual_clusters,list(predictions.values()))


-0.03547185007887272

Run DenStream on SMTP dataset

In [27]:
smpt_dataset = datasets.SMTP()
denstream = cluster.DenStream(decaying_factor=0.01,
                               beta=0.5,
                               mu=2.5,
                               epsilon=0.5,
                               n_samples_init=10)
fp = 0
tp = 0
for x,y in smpt_dataset:
    denstream = denstream.learn_one(x)
    if denstream.predict_one(x) != y:
        fp +=1
    else:
        tp +=1
3


3

In [24]:
t_dict={}
k=0
for i in sorted(set(actual.values())):
    t_dict[i] = k
    k= k+1
actual_clusters=[]
for i in actual.values():
    actual_clusters.append(t_dict[i])
print(format(adjusted_rand_score(list(predictions.values()),actual_clusters),".2f"))
print(tp,fp)

-0.04
95126 30


Run Clustream on SMTP dataset and calculate fp,tp

In [26]:
clustream = cluster.CluStream(
n_macro_clusters=2,
max_micro_clusters=2,
time_gap=100,
    seed=0,
    halflife=0.4
)
fp = 0
tp = 0
for x,y in smpt_dataset:
    clustream = clustream.learn_one(x)
    if clustream.predict_one(x) != y:
        fp +=1
    else:
        tp +=1
print(tp,fp)

523 94633


**References**


1.   https://github.com/online-ml/river/blob/main/river/cluster/k_means.py
2.   https://dl.acm.org/doi/10.1145/3534678.3542600
3.   https://www.section.io/engineering-education/online-machine-learning-with-river-python/#river-python-installation
4.   https://www.youtube.com/watch?v=nzFTmJnIakk
5.   https://riverml.xyz/0.14.0/
6.   https://www.jmlr.org/papers/volume22/20-1380/20-1380.pdf
7.   https://dl.acm.org/doi/pdf/10.1145/3534678.3542600

